In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import janestreet

import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sklearn.metrics import log_loss, roc_auc_score, f1_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

In [ ]:
DATA_PATH = '../input/jane-street-market-prediction/'

BATCH_SIZE = 8192
EPOCHS = 200
LR = 1e-4
WEIGHT_DECAY = 1e-5
#EARLYSTOP_NUM = 3
nFOLDS = 5

training = True
print('Reading input data...')
train = pd.read_csv(f'{DATA_PATH}train.csv',nrows=5000)
train = train.query('date > 85').reset_index(drop=True)
print('Optimizing memory usage...')
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns})
print('Complete')

feat_cols = [f'feature_{i}' for i in range(130)]
target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

all_feat_cols = [col for col in feat_cols]
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

f_mean = np.load('../input/f-mean/f_mean.npy')
f_mean = np.insert(f_mean,0,0)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(132)
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

In [ ]:
device = torch.device("cuda")
model_list = {}
weights = {'M_1':0.5,'M_3':0.5}

for _fold in range(nFOLDS):
    if _fold == 1 or _fold == 3:
        model_name = f"M_{_fold}"
        torch.cuda.empty_cache()
        model = Model().to(device)
        model.load_state_dict(torch.load("../input/resnetoverfit/online_model_v3_{}.pkl".format(_fold)))
        model.eval()
        model_list[model_name] = {"model":model,"weight":weights[model_name]}

In [ ]:
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
for (test_df,pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        

        x_tt = test_df.loc[:, feat_cols].values
        if np.isnan(x_tt.sum()):
             x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean

        cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
        cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
        feature_inp = np.concatenate((
            x_tt,
            np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
            np.array(cross_1_2).reshape(x_tt.shape[0], 1),
        ), axis=1)
            
        pred = np.zeros((1, len(target_cols)))
        for model in model_list:
            pred+=model_list[model]['model'](torch.tensor(feature_inp,dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() * model_list[model]['weight']
        #pred = neutralize(test_df,pred,features)    
        pred = np.median(pred)
        pred_df.action = np.where(pred >= 0.503,1,0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)